# This code uses the kplr interface to download raw light curves from MAST servers

In [ ]:
#I begin by importing the needed packages
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy as sci
import kplr
from astropy.stats import LombScargle
from numpy.polynomial import Chebyshev as T


In [ ]:
#Here, I begin to define all the periferical functions that I'll need. 
#-##############################################################################
#-##############################################################################
# function responsible to get the light curves from MAST server
def get_object(Object,ID,Qtin,Qtfin,method):

    client = kplr.API()
    if Object == "planet":
        box = client.planet(ID)

    if Object == "star":
        box = client.star(ID)

    lcs = box.get_light_curves()
    lcs = lcs[Qtin:Qtfin]

    time, flux_a, flux_c, err_a, err_b = [], [], [], [], []
    s = 0
    for lc in lcs:
        with lc.open() as f:
            # get the data of each file.
            hdu_data = f[1].data
            # here, I set the time, flux and error of each quarter
            time = np.append(time, hdu_data["time"])
            flux_a = np.append(flux_a, hdu_data["pdcsap_flux"])
            err_a  = np.append(err_a,  hdu_data["pdcsap_flux_err"])
            # here is just to set the minimal cadence if I need.
            if s == 0:
                cadencia = (max(time)-min(time))/len(time)
                s = 1

            # depending of the choose normalization:
            flux_div, err_div = normalize(hdu_data["pdcsap_flux"], hdu_data["pdcsap_flux_err"], method)
            flux_c = np.append(flux_c, flux_div)
            err_b =  np.append(err_b, err_div)
    return time, flux_a, flux_c, err_a, err_b

def normalize(flux,err,method):
    """ Normalize the entry flux using the subtraction or division method, choose as "sub" or "div"."""
    onde = 1e8# 671688 #where I want to normalize in case of sub (sum)

    if method == "div":
        mediaF = np.nanmedian(flux)
        mediaE = np.nanmedian(err)
        flux = flux/mediaF
        err = err/mediaE - 1

    if method == "sub":
        mediaF = np.nanmedian(flux)
        flux = flux + (onde - mediaF)

    return flux, err

# From now on, all the comments are going to stay in portuguese.
def reduction (time_nan, flux_nan, err, cadencia, factor=5, norm=1):
    
    #agora vou encontrar as gaps entre os quarters e outras gaps menores e preencher com um determinado ruido
    faltante_time = [] #esta lista guardara o tempo que falta nos gaps
    i = 1       #comecarei a partir do 1 para pegar o segundo comeco (que eh do segundo quarter) pois o primeiro nao interessa
    # neste for, irei conferir cada intervalo entre os pontos fotometricos para conferir se ha alguma pausa maior do que um determinado tempo
    #de observacao, se houver, entao devo preencher essa gap com um determinado ruido
    for i in range (len(time_nan)-1):
        teste = time_nan[i]-time_nan[i-1] #este teste confere o valor de tempo entre dois pontos fotometricos
        # se o intervalo entre os pontos for maior do que a cadencia definida da observacao, entao entro no if para preencher esse espaco
        if teste > 0.0405:
            valor = time_nan[i]-time_nan[i-1] #guarda o intervalo de tempo que precisarei preencher
            falta_time = np.linspace(time_nan[i-1],time_nan[i],(valor/cadencia)) #cria uma lista que contem os horarios que deveriam haver observacao baseado na cadencia predefinida
            faltante_time = np.append(faltante_time, falta_time) #inclui esta lista na minha lista de intervalos das gaps

    faltante_flux = np.zeros(len(faltante_time))+norm # crio uma lista com a mesma quantidade de pontos que o tempo que falto nos gaps normalizados em norm (padrao = 1)
    faltante_err = np.zeros(len(faltante_time))
    faltante_flux = np.random.normal(faltante_flux,np.std(flux_nan)/factor) #aplico alguma randomicidade nesse fluxo
    time_nan = np.append(time_nan, faltante_time) #incluo o tempo sintetico dos gaps no tempo original
    flux_nan = np.append(flux_nan, faltante_flux) #incluo o fluxo sintetico dos gaps no fluxo original
    err_nan = np.append(err, faltante_err)

    time_nan, flux_nan = zip(*sorted(zip(time_nan, flux_nan))) #organizo de forma temporal minha lista total de tempo e fluxo
    time_nan = np.array(time_nan)   #garanto o aspecto de array para o tempo
    flux_nan = np.array(flux_nan) #garanto o aspecto de array para o fluxo flux

    return time_nan, flux_nan, err_nan

def detrending(time,flux, err, pl):

    fluxTrue = np.isfinite(flux) #confere se em um index tem um numero ou nao. retorna True ou False
    index = np.where(fluxTrue == True)[0] #indexs que possuem numeros (True)
    flux_nan = flux[index] #todos os valores de fluxo sem os "nan"s
    time = time[index]          # o tempo que corresponde ao fluxo sem "nan"
    err_nan = err[index]
    time_nan = (time - min(time))   # normaliza o inicio do tempo em zero dias
    # aqui eu coloco em ordem numerica temporal a curva de luz total
    time_nan, flux_nan = zip(*sorted(zip(time_nan, flux_nan)))
    time_nan = np.array(time_nan)   #garanto o aspecto de array para o tempo
    flux_nan = np.array(flux_nan) #garanto o aspecto de array para o fluxo flux


    p = T.fit(time_nan, flux_nan, pl) # faco aqui uma fitagem da curva de forma polinomial com o polinomio definido
    flux_model = p(time_nan)        #aplico a minha fitagem com o tempo de observacao
    flux_detrended = (flux_nan-flux_model)   # faco a subtracao do real com o modelo
    flux_detrended = flux_detrended + 1       # como isso vai pra zero, normalizo de volta em 1

    return time_nan, flux_nan, flux_model, flux_detrended, err_nan

def detrending2(time,flux, err, pl):

    p = T.fit(time, flux, pl) # faco aqui uma fitagem da curva de forma polinomial com o polinomio definido
    flux_model = p(time)        #aplico a minha fitagem com o tempo de observacao
    flux_detrended = (flux-flux_model)   # faco a subtracao do real com o modelo
    flux_detrended = flux_detrended + 1       # como isso vai pra zero, normalizo de volta em 1

    return time, flux, flux_model, flux_detrended, err

def binagem (time_done, flux_done, err, period, nbins):
    foldtime = time_done/period
    foldtime = foldtime % 1
    width = 1.0/nbins #tamanho de cada bin

    bins = np.zeros(nbins)
    weights = np.zeros(nbins)

    for i in range(len(flux_done)):
        n = int(foldtime[i] / width)
        weight = err[i]**-2.0
        bins[n] += flux_done[i]*weight
        weights[n] += weight

    bins /= weights

    binErr = np.sqrt(1.0/(weights))
    binEdges = (np.arange(nbins)*width)
    binEdges = np.linspace(0,period*24,nbins)

    binEdges = binEdges
    plt.plot(binEdges, bins,"og")
    #plt.errorbar(binEdges,bins,yerr=binErr,linestyle='none',marker='o')  # plot binned lightcurve
    plt.show()

#-##############################################################################
#-##############################################################################

# Here is some usefull data:

In [ ]:
Object = "planet"
ID = "kepler-8b"
Qtin = 4
Qtfin = 7
method = "div"
detrendy = 1
preencher = 50
binn = 4000

In [ ]:
# Here, I use all the pre-build funcitions: 

time, flux_original, flux, err_original, err = get_object(Object, ID, Qtin, Qtfin, method)
time_nan, flux_nan, flux_model, flux_detrended, err = detrending(time,flux, err, detrendy)
time_done, flux_done, err = reduction(time_nan,flux_detrended,err,preencher,factor=4)

In [ ]:
#Plot of those reductions
%matplotlib notebook
plt.subplot(3, 1, 1)
plt.title("Original Light Curve, Normalized and detrended")
plt.plot(time, flux_original, 'kx')
plt.subplot(3, 1, 2)
plt.plot(time_nan,flux_nan,'kx')
plt.plot(time_nan,flux_model,'b-',lw=1.5)
plt.ylim(0.99,1.01)
plt.subplot(3, 1, 3)
plt.plot(time_nan, flux_detrended, 'kx')
plt.ylim(0.99,1.01)
plt.show()

In [ ]:
# define the frequencies to search.
%matplotlib notebook
frequency = np.linspace(0.01, 1, 1000)
power = LombScargle(time_done, flux_done).power(frequency)
#frequency, power = LombScargle(time_done, flux_done).autopower()
plt.plot((1/frequency),(power))
plt.xlim(0,50)
#plt.xscale("log")
plt.show()

In [ ]:
#-#########################################
period = 3.5261

foldtime = time_done/period
foldtime = foldtime % 1

In [ ]:
%matplotlib notebook
plt.subplot(2, 1, 1)
plt.ylim(0.99,1.01)
plt.plot(time_done, flux_done,'rx')
plt.subplot(2, 1, 2)
plt.plot(foldtime, flux_done, 'kx')
plt.ylim(0.99,1.01)
plt.show()

In [ ]:
%matplotlib notebook
binagem(time_done,flux_done,err,period,binn)